In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import os.path
import pandas as pd

In [59]:
lista_acoes = ['PETR4', 'MGLU3', 'AMER3', 'OIBR3']

In [63]:
servico = Service(ChromeDriverManager().install())
lista_acoes_validas = []
for acao in lista_acoes:
    navegador = webdriver.Chrome(service=servico)
    navegador.maximize_window()

    navegador.get(f"https://investidor10.com.br/acoes/{acao}/")

    tabela_resultados = navegador.find_element(By.ID, "results_table")
    # desse a tela ate o elemento
    navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)
    time.sleep(2)
    # verifica se ta visivil
    while not tabela_resultados.is_displayed():
        navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)

    time.sleep(2)

    botoes = tabela_resultados.find_element(By.CLASS_NAME, "nav")
    botoes = botoes.find_elements(By.TAG_NAME, "li")


    bt_trimestral = botoes[3]
    bt_trimestral.click()
    time.sleep(1)

    dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
    dropdow = dropdow.find_elements(By.TAG_NAME, "li")
    dropdow[1].click()
    time.sleep(1)

    bt_valores = botoes[1]
    bt_valores.click()
    time.sleep(0.5)
    # clica no botao onde tem o texto "Valores Detalhados"
    dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
    dropdow = dropdow.find_elements(By.TAG_NAME, "li")
    dropdow[1].click()

    time.sleep(1)

    tabela = navegador.find_element(By.ID, "table-balance-results")

    linhas = tabela.find_elements(By.TAG_NAME, "tr")



    # A partir daqui, os dados estao sendo coletados
    # salve esses dados em um arquivo CSV
    colunas = linhas[0].find_elements(By.TAG_NAME, "th")
    if not os.path.isfile('dataset_ML.csv'): # Para nao escrever as colunas novamente
        with open('dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
            f.write("Empresa;Resultado;")
            for coluna in colunas:
                if coluna.text == "" or coluna.text == "#":
                    continue
                # inserindo uma condiçao para nao colocar o ";" na ultima coluna
                if coluna.text == "1T2019":
                    linha = coluna.text + "\n"
                else:
                    linha = coluna.text + ";"
                f.write(linha)
                print(coluna.text, end=" ")
            print()
            f.close()
    with open('dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
        indices_desejados = [1,4,6,9,13]
        for idx, linha in enumerate(linhas[1:], start=1):
            if idx in indices_desejados:
                colunas = linha.find_elements(By.TAG_NAME, "td")
                i = 1
                if len(colunas) !=60: #Significa que estão faltando trimestres nos dados da ação
                    break #Passo para a próxima ação
                f.write(acao + ";")
                if acao not in lista_acoes_validas:
                    lista_acoes_validas.append(acao)
                for coluna in colunas:
                    
                    if coluna.text == "":
                        continue
                    #faça uma condicao que verifique coluna.text é o ultimo elemento e nao coloque o ";"
                    
                    if i == 21:
                        linha = coluna.text + "\n"
                    else:
                        linha = coluna.text + ";"
                    i += 1
                    f.write(linha)

                    print(coluna.text, end=" ")
        
                print()

lista_acoes = lista_acoes_validas

4T2023 3T2023 2T2023 1T2023 4T2022 3T2022 2T2022 1T2022 4T2021 3T2021 2T2021 1T2021 4T2020 3T2020 2T2020 1T2020 4T2019 3T2019 2T2019 1T2019 
Receita Líquida - (R$) 134.258.000.000 124.828.000.000 113.840.000.000 139.068.000.000 158.579.000.000 170.076.000.000 170.960.000.000 141.641.000.000 134.190.000.000 121.594.000.000 110.710.000.000 86.174.000.000 74.972.000.000 70.730.000.000 50.898.000.000 75.470.000.000 72.627.978.000 77.051.002.000 72.566.996.000 79.999.000.000 
Lucro Líquido - (R$) 31.163.000.000 26.760.000.000 28.936.000.000 38.307.000.000 43.502.000.000 46.236.000.000 54.484.000.000 44.783.000.000 31.723.000.000 31.224.000.000 43.041.000.000 1.276.000.000 60.452.000.000 -1.669.000.000 -2.813.000.000 -49.720.000.000 8.538.000.000 8.840.000.000 19.352.000.000 4.240.000.000 
EBIT - (R$) 39.628.000.000 47.546.000.000 41.971.000.000 60.197.000.000 57.811.000.000 74.612.000.000 96.434.000.000 65.398.000.000 58.289.000.000 66.457.000.000 52.181.000.000 33.904.000.000 68.009.000.00

In [65]:
lista_acoes

['PETR4', 'MGLU3', 'OIBR3']

In [66]:
def create_pivot_table(df, index_col, columns_col, reindex_list):
    pivot_table = df.pivot_table(index=index_col, columns=columns_col, aggfunc='sum')
    pivot_table = pivot_table.reindex(reindex_list)
    pivot_table.columns = ['_'.join(col).strip() for col in pivot_table.columns.values] # Transformando o multi index da coluna em um index simples
    pivot_table = pivot_table.filter(like='23', axis=1) # Filtrando apenas os resultados do ano de 2023
    pivot_table = pivot_table.drop(pivot_table.filter(like='4T').columns, axis=1) # Removendo os resultados do 4T de 2023
    return pivot_table

"""Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para 
reindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre, filtrando pelo ano de 2023, que servirá como base de dados para análise"""

'Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para \nreindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre, filtrando pelo ano de 2023, que servirá como base de dados para análise'

In [68]:
def criacao_dataframe_dados_historicos(diretorio, lista_acoes):
    dataframes = []
    for acao in lista_acoes:
        df = pd.read_csv(f'{diretorio}/{acao} Dados Históricos.csv')
        #Inserindo o nome da empresa na coluna "Empresa" e colocando ela como index
        df['Empresa'] = acao
        df.set_index('Empresa', inplace=True)
        # Selecionando apenas o mes de novembro de 2023
        df = df[df['Data'] == '01.11.2023']
        # Calculando a variação - convertendo os valores em float
        df['Último'] = df['Último'].str.replace(',', '.').astype(float)
        df['Abertura'] = df['Abertura'].str.replace(',', '.').astype(float)
        df['Variação'] = (df['Último'] - df['Abertura']) / df['Abertura'] * 100
        # Criando a classificação, que vai servir como o Y do modelo
        if (df['Variação'] > 0).any():
            if (df['Variação'] > 10).any():
                df['Classificação'] = 2
            else:
                df['Classificação'] = 1
        else:
            df['Classificação'] = 0
        dataframes.append(df)
    df_final = pd.concat(dataframes)
    return df_final

"""Essa função está selecionando os arquivos csv dentro da pasta 'dados historicos' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, 
selecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, 
e se subiu mais de 10%, a classificação é 2."""

'Essa função está selecionando os arquivos csv dentro da pasta \'dados historicos\' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, \nselecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, \ne se subiu mais de 10%, a classificação é 2.'

In [69]:
def atribuir_classificacao(dataframe_resultados, dataframe_dados_historicos):
    dataframe_final = dataframe_resultados
    dataframe_final['Classificação'] = dataframe_dados_historicos['Classificação']
    return dataframe_final



Sobre as colunas do dataset:
- **Receita Líquida:** é a quantia de dinheiro que uma empresa ganha com suas vendas após a dedução de devoluções, descontos e impostos sobre vendas.
- **Lucro Líquido:** representa o lucro de uma empresa após todas as despesas e impostos terem sido deduzidos da receita total.
- **EBIT:** representa a lucratividade de uma empresa antes da consideração dos juros e impostos.
- **Dívida Líquida:** a diferença entre todos os compromissos financeiros pendentes e os recursos financeiros imediatamente disponíveis para liquidá-los.
- **ROE:** é uma medida que revela a capacidade de uma empresa gerar lucro a partir do dinheiro que os acionistas investiram nela.


In [72]:


# Ler o arquivo CSV
df = pd.read_csv('dataset_ML.csv', delimiter=';')
df2 = df.copy()


In [73]:
dados = create_pivot_table(df2, 'Empresa', 'Resultado', lista_acoes)
dados

,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),3T2023_Dívida Líquida - (R$),3T2023_EBIT - (R$),3T2023_Lucro Líquido - (R$),3T2023_ROE - (%),3T2023_Receita Líquida - (R$)
Empresa,,,,,,,,,,,,,,,
PETR4,204.125.000.000,60.197.000.000,38.307.000.000,"9,49 %",139.068.000.000,218.390.000.000,41.971.000.000,28.936.000.000,"7,76 %",113.840.000.000,238.304.000.000,47.546.000.000,26.760.000.000,"6,90 %",124.828.000.000
MGLU3,5.038.335.000,16.328.000,-391.221.000,"-3,80 %",9.067.334.000,4.744.924.000,-35.923.000,-301.746.000,"-3,01 %",8.572.256.000,4.118.464.000,661.189.000,-498.330.000,"-5,23 %",8.578.818.000
OIBR3,20.956.798.000,-102.677.000,-1.267.087.000,"-5,47 %",2.535.792.000,21.232.781.000,-266.082.000,-844.524.000,"-3,53 %",2.454.171.000,22.719.925.000,-803.758.000,-2.830.492.000,"-10,55 %",2.421.776.000


In [74]:
df_final = criacao_dataframe_dados_historicos('dados historicos', lista_acoes)
df_final

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Variação,Classificação
Empresa,,,,,,,,,
PETR4,01.11.2023,35.91,33.73,"36,03","32,66","992,20M","3,37%",6.463089,1
MGLU3,01.11.2023,2.02,1.33,"2,35","1,31","4,53B","51,88%",51.879699,2
OIBR3,01.11.2023,0.65,0.60,"0,75","0,59","120,11M","10,17%",8.333333,1


In [75]:
df_classificado = atribuir_classificacao(dados, df_final)
df_classificado

,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),3T2023_Dívida Líquida - (R$),3T2023_EBIT - (R$),3T2023_Lucro Líquido - (R$),3T2023_ROE - (%),3T2023_Receita Líquida - (R$),Classificação
Empresa,,,,,,,,,,,,,,,,
PETR4,204.125.000.000,60.197.000.000,38.307.000.000,"9,49 %",139.068.000.000,218.390.000.000,41.971.000.000,28.936.000.000,"7,76 %",113.840.000.000,238.304.000.000,47.546.000.000,26.760.000.000,"6,90 %",124.828.000.000,1
MGLU3,5.038.335.000,16.328.000,-391.221.000,"-3,80 %",9.067.334.000,4.744.924.000,-35.923.000,-301.746.000,"-3,01 %",8.572.256.000,4.118.464.000,661.189.000,-498.330.000,"-5,23 %",8.578.818.000,2
OIBR3,20.956.798.000,-102.677.000,-1.267.087.000,"-5,47 %",2.535.792.000,21.232.781.000,-266.082.000,-844.524.000,"-3,53 %",2.454.171.000,22.719.925.000,-803.758.000,-2.830.492.000,"-10,55 %",2.421.776.000,1
